In [1]:
import numpy as np
import tensorflow.keras
import torch
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv1D, AveragePooling1D, Conv2D, MaxPooling2D,ReLU
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model #save and load models
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
import IPython.display as ipd
from kymatio import Scattering1D
import hitdifferentparts
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pescador
import random
import os
import librosa

In [2]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# scattering order one
def getsc_new(y,J,Q_num,order):
    """
    this function outputs scattering transform of a time-domain signal.
    """
    N = len(y)
    scattering = Scattering1D(J = J,shape=(N,), Q = Q_num, max_order=order)
    Sy = scattering(torch.Tensor(y))
    return Sy

In [4]:
def feature_sampler(df,params_normalized,idx,path_to_folder,J,Q,order):
    """
    output a {input, ground truth} pair for the designated audio sample
    """
    i=idx
    y=np.array(params_normalized[i,:]).reshape((5,)) #df.values[i,1:-1]
    path_to_audio = os.path.join(path_to_folder,str(df.values[i,0])+"_sound.wav") 
    x,fs=librosa.load(path_to_audio)
    Sy = getsc_new(x,J,Q,order)
    m,n = Sy.shape
    Sy2 = np.array(Sy).reshape((n,m))
    
    while True:
        yield {'input': Sy2,'y': y}

        
def data_generator(df, params_normalized, path_to_folder, J, Q, order, batch_size, idx, active_streamers,
                        rate, random_state=12345678):
    """
    use streamers to output a batch of {input groundtruth} pairs. 
    """
    seeds = []
    for i in idx:
        streamer = pescador.Streamer(feature_sampler, df, params_normalized, i,path_to_folder,J,Q,order)
        seeds.append(streamer)

    # Randomly shuffle the seeds
    random.shuffle(seeds)

    mux = pescador.StochasticMux(seeds, active_streamers, rate=rate, random_state=random_state)
   
    if batch_size == 1:
        return mux
    else:
        return pescador.maps.buffer_stream(mux, batch_size)



In [27]:
def create_model(J,Q,order,path_to_train,k_size):
    fname = random.choice(os.listdir(path_to_train))
    rand_audio = os.path.join(path_to_train,fname)
    y,sr = librosa.load(rand_audio)
    Sy = getsc_new(torch.Tensor(y),J,Q,order).T
    nrow, ncol = Sy.shape 
    naudio = batch_size         # number of images in batch
    nchan_in = 1       # number of input channels.  1 since it is BW
    #input_shape = (nrow,ncol,nchan_in)  # input shape of 1 image
    input_shape = Sy.shape
    batch_shape = (naudio,nrow,ncol,nchan_in)  # shape of image batch
    #x = Sy.reshape(batch_shape)
    kernel_size = (k_size,)
    nchan_out = 16

    K.clear_session()
    model=Sequential()
    #1 conv layer +  1 batch normalization + nonlinear activation + pooling
    model.add(Conv1D(input_shape=input_shape, filters=nchan_out,
                     kernel_size=kernel_size, padding="same",name='conv1'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(AveragePooling1D(pool_size=(4,)))
    
   #second time
    model.add(Conv1D(filters=16,
                     kernel_size=kernel_size, padding="same",name='conv2' ))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(AveragePooling1D(pool_size=(4,)))
    
    #third time
    model.add(Conv1D(filters=16,
                     kernel_size=kernel_size, padding="same",name='conv3' ))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(AveragePooling1D(pool_size=(4,)))
    
    #fourth time
    model.add(Conv1D(filters=16,
                     kernel_size=kernel_size, padding="same",name='conv4' ))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(AveragePooling1D(pool_size=(2,)))

    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    #what activation should be chosen for last layer, for regression problem? should be a linear function
    model.add(Dense(5, activation='linear')) #output layer that corresponds to the 5 physical parameters.


    # Compile the model
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
    print(model.summary())
    return model

In [21]:
def make_generator(batch_size,active_streamers,J,Q,order,random_state):
    df_train = pd.read_csv("./train_param.csv")
    df_test = pd.read_csv("./test_param.csv")
    df_val = pd.read_csv("./val_param.csv")
    df_full = pd.read_csv("./diffshapes_param.csv")
    
    # normalization of the physical parameters
    params = df_train.values[:,1:-1]
    scaler = MinMaxScaler()
    scaler.fit(params)
    train_params_normalized = scaler.transform(params)
    
    train_idx = np.arange(0,params.shape[0],1)#np.arange(0,1000,1) #df_train.values[:1000,0]
    test_idx = np.arange(0,df_test.values.shape[0],1) #df_test.values[:300,0]
    val_idx = np.arange(0,df_val.values.shape[0],1)

   
    test_params_normalized = scaler.transform(df_test.values[:,1:-1])
    val_params_normalized = scaler.transform(df_val.values[:,1:-1])
    train_batches=data_generator(df_train,train_params_normalized, path_to_train,J, Q, order, batch_size, train_idx,active_streamers,rate=64,random_state=random_state)
    test_batches=data_generator(df_test,test_params_normalized, path_to_test,J, Q, order, batch_size, test_idx,active_streamers,rate=64,random_state=random_state)
    val_batches = data_generator(df_val,val_params_normalized, path_to_val,J, Q, order, batch_size, val_idx,active_streamers,rate=64,random_state=random_state)
    steps_per_epoch = len(train_idx) // batch_size
    
    train_gen = pescador.maps.keras_tuples(train_batches, 'input', 'y')
    test_gen = pescador.maps.keras_tuples(test_batches, 'input', 'y')
    val_gen = pescador.maps.keras_tuples(val_batches, 'input', 'y')
    
    return train_gen,test_gen,val_gen,scaler,steps_per_epoch


In [7]:
import pickle
def get_val_from_pickle(scaler):
    pkl_path = '/scratch/hh2263/drum_data/val/J_8_Q_1_order_2.pkl'
    pkl_file = open(pkl_path, 'rb')
    Sy_val,y_val = pickle.load(pkl_file) 
    Sy_val = Sy_val.reshape((Sy_val.shape[2],Sy_val.shape[0],Sy_val.shape[1]))
    y_val = y_val.astype('float32')
    y_val_normalized = scaler.transform(y_val)
    return Sy_val,y_val_normalized

In [8]:
def fit_model(epochs,train_gen,Sy_val,y_val_normalized,steps_per_epoch):
    output_dir = "../output/"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model_filepath = os.path.join(output_dir, "training_1/cp.ckpt")
    log_filepath = os.path.join(output_dir, 'train_log.csv')

    callbacks = []
    callbacks.append(EarlyStopping(patience=10))
    callbacks.append(ModelCheckpoint(model_filepath, save_best_only=True))
    callbacks.append(CSVLogger(log_filepath))
    
    for epoch in range(epochs):
        model.fit(train_gen,
                  steps_per_epoch=steps_per_epoch/5,
                  epochs=1,
                  use_multiprocessing=True,
                  callbacks = callbacks,
                  verbose=1,

                 )
        print('done fitting')
        loss,accuracy = model.evaluate(Sy_val,y_val_normalized)
        #print(loss,accuracy)


In [11]:
epochs=12
batch_size=32
random_state=12345678
active_streamers=64
J = 8
Q = 1
order = 2 # remember to go to order 2 eventually
path_to_train = "/scratch/hh2263/drum_data/train/"
path_to_test = "/scratch/hh2263/drum_data/test/"
path_to_val = "/scratch/hh2263/drum_data/val"

model = create_model(J,Q,order,path_to_train,k_size=8)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv1D)               (None, 128, 16)           5520      
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 16)           64        
_________________________________________________________________
activation (Activation)      (None, 128, 16)           0         
_________________________________________________________________
average_pooling1d (AveragePo (None, 32, 16)            0         
_________________________________________________________________
conv2 (Conv1D)               (None, 32, 16)            2064      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 16)            64        
_________________________________________________________________
activation_1 (Activation)    (None, 32, 16)            0

In [22]:
train_gen,test_gen,val_gen,scaler,steps_per_epoch =make_generator(batch_size,active_streamers,J,Q,order,random_state)
Sy_val,y_val_normalized = get_val_from_pickle(scaler)

## train with steps_per_epoch/5 - overfit after first epoch

In [13]:
output_dir = "../output/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
model_filepath = os.path.join(output_dir, "training_1/cp.ckpt")
log_filepath = os.path.join(output_dir, 'train_log.csv')

callbacks = []
callbacks.append(EarlyStopping(patience=10))
callbacks.append(ModelCheckpoint(model_filepath, save_best_only=True))
callbacks.append(CSVLogger(log_filepath))

for epoch in range(epochs):
    model.fit(train_gen,
              steps_per_epoch=steps_per_epoch/5,
              epochs=1,
              use_multiprocessing=True,
              callbacks = callbacks,
              verbose=1,

             )
    print('done fitting')
    loss,accuracy = model.evaluate(Sy_val,y_val_normalized)
    #print(loss,accuracy)

514/513 [==============================] - 206s 401ms/step - loss: 0.0708 - acc: 0.6124
done fitting
514/513 [==============================] - 211s 411ms/step - loss: 0.0322 - acc: 0.6853
done fitting
514/513 [==============================] - 210s 408ms/step - loss: 0.0278 - acc: 0.7097
done fitting
514/513 [==============================] - 201s 391ms/step - loss: 0.0250 - acc: 0.7259
done fitting
7776/7776 [==============================] - 2s 221us/sample - loss: 75.4812 - acc: 0.0521


Process Keras_worker_ForkPoolWorker-5:
Traceback (most recent call last):
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 832, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/site-packages/pescador/maps.py", line 228, in keras_tuples
    for data in stream:
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/site-packages/pescador/maps.py", line 130, in buffer_stream
    for item in stream:
  File "/home

KeyboardInterrupt: 

Process Keras_worker_ForkPoolWorker-6:
Traceback (most recent call last):
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/conne

## decrease steps_per_epoch - slowed down val loss increase

In [24]:
model_2 = create_model(J,Q,order,path_to_train,k_size=8)
train_gen,test_gen,val_gen,scaler,steps_per_epoch=make_generator(batch_size=64,
                                                                 active_streamers=64,
                                                                 J=J,
                                                                 Q=Q,
                                                                 order=order,
                                                                 random_state=random_state)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv1D)               (None, 128, 16)           5520      
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 16)           64        
_________________________________________________________________
activation (Activation)      (None, 128, 16)           0         
_________________________________________________________________
average_pooling1d (AveragePo (None, 32, 16)            0         
_________________________________________________________________
conv2 (Conv1D)               (None, 32, 16)            2064      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 16)            64        
_________________________________________________________________
activation_1 (Activation)    (None, 32, 16)            0

In [25]:
output_dir = "../output/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
model_filepath = os.path.join(output_dir, "cp.ckpt")
log_filepath = os.path.join(output_dir, 'train_log.csv')

callbacks = []
callbacks.append(EarlyStopping(patience=10))
callbacks.append(ModelCheckpoint(model_filepath, save_best_only=True))
callbacks.append(CSVLogger(log_filepath))

for epoch in range(epochs):
    model_2.fit(train_gen,
              steps_per_epoch=200,
              epochs=1,
              validation_data=(Sy_val,y_val_normalized),
              use_multiprocessing=True,
              callbacks = callbacks,
              verbose=1,

             )
    print('done fitting')
    #loss,accuracy = model.evaluate(Sy_val,y_val_normalized)
    #print(loss,accuracy)

200/200 [==============================] - 167s 836ms/step - loss: 0.0914 - acc: 0.5772 - val_loss: 0.2050 - val_acc: 0.3373
done fitting
200/200 [==============================] - 163s 815ms/step - loss: 0.0378 - acc: 0.6504 - val_loss: 0.8790 - val_acc: 0.3381
done fitting
200/200 [==============================] - 177s 885ms/step - loss: 0.0336 - acc: 0.6672 - val_loss: 2.0861 - val_acc: 0.3112
done fitting
200/200 [==============================] - 177s 883ms/step - loss: 0.0309 - acc: 0.6659 - val_loss: 8.1088 - val_acc: 0.0472
done fitting
200/200 [==============================] - 174s 871ms/step - loss: 0.0279 - acc: 0.6955 - val_loss: 8.4053 - val_acc: 0.3167
done fitting
188/200 [===========================>..] - ETA: 10s - loss: 0.0245 - acc: 0.7106

Process Keras_worker_ForkPoolWorker-26:
Traceback (most recent call last):
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))


KeyboardInterrupt: 

  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 832, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/site-packages/pescador/maps.py", line 228, in keras_tuples
    for data in stream:
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/site-packages/pescador/maps.py", line 130, in buffer_stream
    for item in stream:
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/site-packages/pescador/mux.py", line 194, in iterate
    yield six.advance_iterator(active_mux.streams_[idx])
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/site-packages/pescador/core.py", line 204, in iterate
    for n, obj in enumerate(active_streamer.stream_):
  File "<ipython-input-4-57d83eccdc7b>", line 9, in feature_sampler
    Sy = getsc_new(x,J,Q,order)
  File "<ipython-input-3-8735c79db0a2>", line 7, in getsc_new
    scattering

## reduce kernel size


In [28]:
model_3 = create_model(J,Q,order,path_to_train,k_size=4)
train_gen,test_gen,val_gen,scaler,steps_per_epoch=make_generator(batch_size=64,
                                                                 active_streamers=64,
                                                                 J=J,
                                                                 Q=Q,
                                                                 order=order,
                                                                 random_state=random_state)



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv1D)               (None, 128, 16)           2768      
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 16)           64        
_________________________________________________________________
activation (Activation)      (None, 128, 16)           0         
_________________________________________________________________
average_pooling1d (AveragePo (None, 32, 16)            0         
_________________________________________________________________
conv2 (Conv1D)               (None, 32, 16)            1040      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 16)            64        
_________________________________________________________________
activation_1 (Activation)    (None, 32, 16)            0

In [29]:
output_dir = "../output/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
model_filepath = os.path.join(output_dir, "cp2.ckpt")
log_filepath = os.path.join(output_dir, 'train_log.csv')

callbacks = []
callbacks.append(EarlyStopping(patience=10))
callbacks.append(ModelCheckpoint(model_filepath, save_best_only=True))
callbacks.append(CSVLogger(log_filepath))

for epoch in range(epochs):
    model_3.fit(train_gen,
              steps_per_epoch=200,
              epochs=1,
              validation_data=(Sy_val,y_val_normalized),
              use_multiprocessing=True,
              callbacks = callbacks,
              verbose=1,

             )
    print('done fitting')
    #loss,accuracy = model.evaluate(Sy_val,y_val_normalized)
    #print(loss,accuracy)

200/200 [==============================] - 173s 865ms/step - loss: 0.1836 - acc: 0.5273 - val_loss: 0.1487 - val_acc: 0.0012
done fitting
200/200 [==============================] - 164s 820ms/step - loss: 0.0425 - acc: 0.6468 - val_loss: 1.0381 - val_acc: 0.0242
done fitting
200/200 [==============================] - 166s 829ms/step - loss: 0.0342 - acc: 0.6821 - val_loss: 1.7914 - val_acc: 0.3023
done fitting
200/200 [==============================] - 173s 867ms/step - loss: 0.0295 - acc: 0.7073 - val_loss: 10.2718 - val_acc: 0.3057
done fitting


KeyboardInterrupt: 

Process Keras_worker_ForkPoolWorker-37:
Traceback (most recent call last):
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 832, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/site-packages/pescador/maps.py", line 228, in keras_tuples
    for data in stream:
  File "/home/hh2263/miniconda3/envs/drum-1.15/lib/python3.7/site-packages/pescador/maps.py", line 130, in buffer_stream
    for item in stream:
  File "/hom